In [ ]:
from sklearn.datasets import make_regression

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split


In [ ]:
X,y = make_regression(n_samples  = 10000,
                      n_features = 15,
                      noise      = 0.3
                      )

In [ ]:
X.shape, y.shape

((10000, 15), (10000,))

In [ ]:
class CustomDataset(Dataset):

    def __init__(self,X,y):
        self.X = torch.tensor(X, dtype= torch.float32)
        self.y = torch.tensor(y, dtype= torch.float32).view(-1,1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=13)

In [ ]:
train_dataset = CustomDataset(X_train,y_train)
test_dataset  = CustomDataset(X_valid,y_valid)

In [ ]:
train_data_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_data_loader  = DataLoader(test_dataset, batch_size = 64, shuffle = False)

In [ ]:
len(train_data_loader),len(test_data_loader)

(125, 32)

In [ ]:
class TabularRegression(nn.Module):

    def __init__(self,input_dim, output_dim):

        super(TabularRegression,self).__init__()

        self.fc = nn.Linear(input_dim,output_dim, dtype = torch.float32)

    def forward(self,x):
        return self.fc(x)

In [ ]:
model       = TabularRegression(15,1)
criterion   = nn.MSELoss(reduction = 'sum')
optimizer   = torch.optim.SGD(model.parameters(), lr=0.0001)
num_epochs  = 10

In [ ]:
for epoch in range(num_epochs):

    model.train()
    train_loss = 0.0

    for batch in train_data_loader:
        inputs,targets = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss    = criterion(outputs,targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for batch in test_data_loader:
            inputs,targets = batch
            outputs = model(inputs)
            loss = criterion(targets,outputs)
            valid_loss += loss.item()


    train_loss = train_loss/len(train_dataset)
    valid_loss = valid_loss/len(test_dataset)


    print(f'Epoch = {epoch+1}, train loss = {train_loss} and validation loss = {valid_loss}')

Epoch = 1, train loss = 5164.699609863282 and validation loss = 655.0061958007813
Epoch = 2, train loss = 191.37634432983398 and validation loss = 25.020505569458006
Epoch = 3, train loss = 7.373380059719086 and validation loss = 1.0622448148727417
Epoch = 4, train loss = 0.3793655114173889 and validation loss = 0.1283458058834076
Epoch = 5, train loss = 0.10539611965417862 and validation loss = 0.0926223908662796
Epoch = 6, train loss = 0.094540382117033 and validation loss = 0.09151717239618301
Epoch = 7, train loss = 0.09409736174345017 and validation loss = 0.09159369385242462
Epoch = 8, train loss = 0.0940742946267128 and validation loss = 0.09157053434848786
Epoch = 9, train loss = 0.09406748303771019 and validation loss = 0.09148623710870743
Epoch = 10, train loss = 0.09406366309523582 and validation loss = 0.0915279860496521
